In [99]:
import pandas as pd
from tqdm import tqdm
import requests

In [100]:
file_url = "./make_file/(도로폭)_서울시_도로_목록.csv"
file_encofing = "UTF-8"

In [101]:
data = pd.read_csv(file_url, encoding=file_encofing, low_memory=False)

In [102]:
data.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모'],
      dtype='object')

#### kakao api

In [103]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
print(open_api_key_file['사이트'].value_counts())

KAKAO_API = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST_sub'].values[0][1]  

사이트
공공데이터                     1
kakao_api_key_REST        1
kakao_api_key_REST_sub    1
카카오 JS                    1
국토부v-world                1
GOOGLE_MAP                1
행정안전부                     1
Name: count, dtype: int64


In [104]:
def get_gu_from_dong(input_dong_name) :
    dong_name = "서울시 " + input_dong_name
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API}"}
    params = {"query": dong_name}
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API 요청 실패: {response.status_code}, {response.text}")
    
    result = response.json()
    if not result["documents"]:
        raise ValueError("검색 결과 없음")
    
    first_result = result["documents"][0]
    x, y = first_result["x"], first_result["y"]
    
    # 좌표 -> 행정구역 변환
    region_url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    params = {"x": x, "y": y}
    response = requests.get(region_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API 요청 실패: {response.status_code}, {response.text}")
    
    region_result = response.json()
    for region in region_result["documents"]:
        if region["region_type"] == "B":  # 법정동(B) 대신 행정동(H) 확인 가능
            return region["region_2depth_name"]  # 구 정보 반환
    
    raise ValueError("구 정보를 찾을 수 없음")




In [123]:
temp_dong_list = list(data['행정동'].unique())

In [106]:
dong_gu_dictionary = {}

for i in tqdm(range(len(temp_dong_list))) :
    dong_name = temp_dong_list[i]
    gu_name = get_gu_from_dong(dong_name)
    dong_gu_dictionary[dong_name] = gu_name

100%|█████████████████████████████████████████| 463/463 [01:26<00:00,  5.35it/s]


In [107]:
data['행정구역'] = data['행정동']

In [108]:
data['행정구역'] = data['행정구역'].map(dong_gu_dictionary)

In [109]:
data['행정구역'].value_counts()

행정구역
강북구        11411
성북구        10640
은평구         9056
동대문구        8515
강남구         8337
서대문구        8276
강동구         7818
송파구         7704
서초구         7100
중랑구         6843
강서구         6496
광진구         6223
용산구         5996
노원구         5949
구로구         5876
도봉구         5746
성동구         5362
영등포구        5233
마포구         5104
관악구         5063
종로구         4525
양천구         4164
중구          4121
금천구         3498
동작구         2288
안양시 만안구        5
하남시            4
광명시            4
성남시 수정구        3
고양시 덕양구        1
의정부시           1
Name: count, dtype: int64

In [110]:
seoul_districts = [
    "강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구", "노원구", "도봉구",
    "동대문구", "동작구", "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구", "양천구",
    "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"
]


In [111]:
data = data[data['행정구역'].isin(seoul_districts)]
data['행정구역'].value_counts()

행정구역
강북구     11411
성북구     10640
은평구      9056
동대문구     8515
강남구      8337
서대문구     8276
강동구      7818
송파구      7704
서초구      7100
중랑구      6843
강서구      6496
광진구      6223
용산구      5996
노원구      5949
구로구      5876
도봉구      5746
성동구      5362
영등포구     5233
마포구      5104
관악구      5063
종로구      4525
양천구      4164
중구       4121
금천구      3498
동작구      2288
Name: count, dtype: int64

In [119]:
# 25개 구에 대한 빈 데이터프레임 생성
district_dfs = {district: pd.DataFrame(columns=data.columns) for district in seoul_districts}


In [120]:

for i in tqdm(range(len(data))):
    district = data['행정구역'].iloc[i]
    
    # district_dfs[district]가 비어 있는 경우 처리
    if district not in district_dfs or district_dfs[district].empty:
        district_dfs[district] = data.iloc[[i]].copy()  # 기존 DataFrame 복사
        district_dfs[district]['id'] = 1  # id 컬럼 추가, 첫 번째 행의 id는 1
    else:
        district_dfs[district] = pd.concat([district_dfs[district], data.iloc[[i]]], ignore_index=True)
        district_dfs[district]['id'] = range(1, len(district_dfs[district]) + 1)  # id 컬럼에 1부터 증가하는 값 할당

    # id 컬럼을 첫 번째 열로 이동
    cols = ['id'] + [col for col in district_dfs[district].columns if col != 'id']
    district_dfs[district] = district_dfs[district][cols]


100%|█████████████████████████████████| 161344/161344 [01:13<00:00, 2203.00it/s]


In [121]:
# 각 구별 데이터 저장 
for district, df in district_dfs.items():
    try :
        df_len = len(df)
        save_file_url = f"./make_file/gu_division/{district}_{df_len}_도로_데이터.csv"
        df.to_csv(save_file_url, index=False, encoding="utf-8")
        print(f"{save_file_url} 파일 저장 완료")
    except OSError as e :
        print(e)

./make_file/gu_division/강남구_8337_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/강동구_7818_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/강북구_11411_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/강서구_6496_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/관악구_5063_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/광진구_6223_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/구로구_5876_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/금천구_3498_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/노원구_5949_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/도봉구_5746_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/동대문구_8515_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/동작구_2288_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/마포구_5104_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/서대문구_8276_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/서초구_7100_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/성동구_5362_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/성북구_10640_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/송파구_7704_도로_데이터.csv 파일 저장 완료
./make_file/gu_division/양천구_4164_도로_데이터.cs